In [1]:
%%init_spark --stderr
launcher.conf.spark.app.name = "DIC Exercise 2"
launcher.conf.spark.default.parallelism = 12

In [2]:
println("spark.app.name: " + spark.conf.get("spark.app.name"))
println("spark.master: " + spark.conf.get("spark.master"))
println("spark.submit.deployMode: " + spark.conf.get("spark.submit.deployMode"))
println("sc.defaultParallelism: " + sc.defaultParallelism)

Intitializing Scala interpreter ...

Spark Web UI available at http://captain01.os.hpc.tuwien.ac.at:9999/proxy/application_1715326141961_0081
SparkContext available as 'sc' (version = 3.2.3, master = yarn, app id = application_1715326141961_0081)
SparkSession available as 'spark'


spark.app.name: DIC Exercise 2
spark.master: yarn
spark.submit.deployMode: client
sc.defaultParallelism: 12


In [3]:
!hdfs dfs -ls -h /user/dic24_shared/amazon-reviews/full

Found 2 items


-rw-r--r--   3 blee supergroup      55.3 M 2023-03-13 11:18 /user/dic24_shared/amazon-reviews/full/reviews_devset.json


-rw-r--r--   3 blee supergroup      54.3 G 2023-03-13 11:18 /user/dic24_shared/amazon-reviews/full/reviewscombined.json




In [4]:
val reviews = spark.read.json("hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json")

[Stage 0:>                                                         (0 + 1) / 12]
[Stage 0:=========>                                                (2 + 1) / 12]
[Stage 0:========================>                                 (5 + 1) / 12]
[Stage 0:======================================>                   (8 + 2) / 12]
[Stage 0:====================================================>    (11 + 1) / 12]
                                                                                


reviews: org.apache.spark.sql.DataFrame = [asin: string, category: string ... 8 more fields]


In [5]:
reviews.show()

+----------+--------------------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin|            category| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------------------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0981850006|Patio_Lawn_and_Garde|  [6, 7]|    5.0|This was a gift f...| 12 3, 2009|A2VNYWOPJ13AFP|Amazon Customer "...|              Delish|    1259798400|
|B00002N66D|Patio_Lawn_and_Garde|  [1, 1]|    5.0|This is a very ni...| 12 3, 2012|A2E5XXXC07AGA7|               James|       Nice spreader|    1354492800|
|B00002N67U|Patio_Lawn_and_Garde|  [0, 1]|    1.0|The metal base wi...|08 13, 2008|A16PX63WZIEQ13|             Finaldx| Terrible spike base|    1218585600|
|B00002N6AN|Patio_Lawn_and_Garde|  [0, 0]|    4.0|For the most p

In [6]:
val stopwordsSource = scala.io.Source.fromFile("/home/dic24/e12347510/Exercise_2/data/stopwords.txt")

// Convert the Source to a Set for efficient lookup
val stopwordsImported = stopwordsSource.getLines.toSet

// Convert the Set to an RDD
val stopwords = sc.parallelize(stopwordsImported.toSeq)

// Close the Source object to release resources
stopwordsSource.close()

stopwordsSource: scala.io.BufferedSource = <iterator>
stopwordsImported: scala.collection.immutable.Set[String] = Set(serious, latterly, absorbs, looks, particularly, used, e, printer, down, regarding, entirely, regardless, moreover, please, read, ourselves, able, behind, for, despite, s, maybe, viz, further, corresponding, x, any, wherein, across, name, allows, this, instead, in, taste, ought, myself, have, your, off, once, are, is, mon, his, oh, why, rd, knows, bulbs, too, among, course, greetings, somewhat, bibs, everyone, seen, likely, said, try, already, soon, nobody, got, given, song, using, less, am, consider, hence, than, n, accordingly, four, anyhow, want, three, forth, whereby, himself, specify, yes, throughout, inasmuch, but, whether, sure, below, aren, co, best, plus, become...


In [7]:
// Extract the reviewText attribute
val reviewTextsDF = reviews.select("reviewText")

// Convert DataFrame to Array[String]
val reviewTextsArray = reviewTextsDF.map(row => row.getString(0)).collect()

[Stage 2:=========>                                                (2 + 2) / 12]
[Stage 2:========================>                                 (5 + 1) / 12]
[Stage 2:=============================>                            (6 + 1) / 12]
[Stage 2:======================================>                   (8 + 1) / 12]
[Stage 2:===============================================>         (10 + 1) / 12]
                                                                                


reviewTextsDF: org.apache.spark.sql.DataFrame = [reviewText: string]
reviewTextsArray: Array[String] = Array(This was a gift for my other husband.  He's making us things from it all the time and we love the food.  Directions are simple, easy to read and interpret, and fun to make.  We all love different kinds of cuisine and Raichlen provides recipes from everywhere along the barbecue trail as he calls it. Get it and just open a page.  Have at it.  You'll love the food and it has provided us with an insight into the culture that produced it. It's all about broadening horizons.  Yum!!, This is a very nice spreader.  It feels very solid and the pneumatic tires give it great maneuverability and handling over bumps.  The control arm is solid metal, not a cable, which gives you precise contro...


In [8]:
val stopwordsArray = stopwords.collect

stopwordsArray: Array[String] = Array(serious, latterly, absorbs, looks, particularly, used, e, printer, down, regarding, entirely, regardless, moreover, please, read, ourselves, able, behind, for, despite, s, maybe, viz, further, corresponding, x, any, wherein, across, name, allows, this, instead, in, taste, ought, myself, have, your, off, once, are, is, mon, his, oh, why, rd, knows, bulbs, too, among, course, greetings, somewhat, bibs, everyone, seen, likely, said, try, already, soon, nobody, got, given, song, using, less, am, consider, hence, than, n, accordingly, four, anyhow, want, three, forth, whereby, himself, specify, yes, throughout, inasmuch, but, whether, sure, below, aren, co, best, plus, becomes, what, unto, different, would, although, elsewhere, causes, another, certainly...


In [9]:
def tokenizeAndRemoveDuplicates(reviewText: String): Seq[String] = {
    // Split the review text into tokens using regular expression
    val tokens = reviewText.toLowerCase.split("[^a-zA-Z<>^|]+")

    // Remove duplicates
    tokens.distinct.filter(token => token.length > 1 && !stopwordsArray.contains(token))
}

tokenizeAndRemoveDuplicates: (reviewText: String)Seq[String]


In [10]:
// Apply the function to each review text using map
val tokenizedReviewsRDD = reviewTextsArray.map(tokenizeAndRemoveDuplicates)

tokenizedReviewsRDD: Array[Seq[String]] = Array(WrappedArray(gift, husband, making, things, time, love, food, directions, simple, easy, interpret, make, kinds, cuisine, raichlen, recipes, barbecue, trail, calls, open, page, provided, insight, culture, produced, broadening, horizons, yum), WrappedArray(nice, spreader, feels, solid, pneumatic, tires, give, great, maneuverability, handling, bumps, control, arm, metal, cable, precise, long, time, settings, experimentation, products, true, good, distribution, flings, material, farther, side, left, crappy, edgeguard), WrappedArray(metal, base, hose, attachments, poorly, designed, made, previous, reviewer, pointed, leaks, badly, pops, fix, tighten, junction, plastic, wears, bit, falls, sprinkler, useless, avoid, gilmour, sprinklers, spike, hea...


In [11]:
val totalCount = tokenizedReviewsRDD.map(_.length).sum

totalCount: Int = 2123321


In [12]:
// Extract the category attribute
val categoriesDF = reviews.select("category")

// Convert DataFrame to Array[String]
val categoriesArray = categoriesDF.map(row => row.getString(0)).collect()

[Stage 4:===============================================>         (10 + 1) / 12]
                                                                                


categoriesDF: org.apache.spark.sql.DataFrame = [category: string]
categoriesArray: Array[String] = Array(Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn...


In [13]:
// Step 1: Zip categories with tokenized reviews
val categorizedReviews = categoriesArray.zip(tokenizedReviewsRDD)

categorizedReviews: Array[(String, Seq[String])] = Array((Patio_Lawn_and_Garde,WrappedArray(gift, husband, making, things, time, love, food, directions, simple, easy, interpret, make, kinds, cuisine, raichlen, recipes, barbecue, trail, calls, open, page, provided, insight, culture, produced, broadening, horizons, yum)), (Patio_Lawn_and_Garde,WrappedArray(nice, spreader, feels, solid, pneumatic, tires, give, great, maneuverability, handling, bumps, control, arm, metal, cable, precise, long, time, settings, experimentation, products, true, good, distribution, flings, material, farther, side, left, crappy, edgeguard)), (Patio_Lawn_and_Garde,WrappedArray(metal, base, hose, attachments, poorly, designed, made, previous, reviewer, pointed, leaks, badly, pops, fix, tighten, junction, plastic, ...


In [14]:
val categorizedReviewsRDD = sc.parallelize(categorizedReviews)

categorizedReviewsRDD: org.apache.spark.rdd.RDD[(String, Seq[String])] = ParallelCollectionRDD[17] at parallelize at <console>:25


In [15]:
// Step 2: Calculate term frequency
val termFrequencyRDD = categorizedReviewsRDD
  // Flatten the tokens and add category
  .flatMap{ case (category, tokens) => tokens.map(token => ((category, token), 1)) }
  // Aggregate by category and term
  .reduceByKey(_ + _)

termFrequencyRDD: org.apache.spark.rdd.RDD[((String, String), Int)] = ShuffledRDD[19] at reduceByKey at <console>:29


In [16]:
// Step 3: Calculate term categories
val termCategoriesRDD = termFrequencyRDD
  // Aggregate by term and category, also calculate term total
  .map{ case ((category, term), count) => (term, (category, count)) }
  .groupByKey()
  .flatMap{ case (term, counts) =>
    // Calculate term_total
    val termTotal = counts.map(_._2).sum
    // Calculate (A, B)
    counts.map{ case (category, count) => ((category, term), (count, termTotal - count)) }
  }

termCategoriesRDD: org.apache.spark.rdd.RDD[((String, String), (Int, Int))] = MapPartitionsRDD[22] at flatMap at <console>:29


In [17]:
val totalCategory = categorizedReviewsRDD
  .map { case (category, _) => (category, 1) }
  .reduceByKey(_ + _)
  .collectAsMap()

24/05/11 19:39:41 WARN TaskSetManager: Stage 5 contains a task of very large size (1815 KiB). The maximum recommended task size is 1000 KiB.

[Stage 5:>                                                         (0 + 2) / 12]
[Stage 5:====>                                                     (1 + 2) / 12]
[Stage 5:==============>                                           (3 + 2) / 12]
[Stage 5:========================>                                 (5 + 2) / 12]
[Stage 5:======================================>                   (8 + 2) / 12]
                                                                                


totalCategory: scala.collection.Map[String,Int] = Map(Kindle_Store -> 3205, Electronic -> 7825, Automotive -> 1374, Pet_Supplie -> 1235, Clothing_Shoes_and_Jewelry -> 5749, Baby -> 916, Grocery_and_Gourmet_Food -> 1297, Musical_Instrument -> 500, Movies_and_TV -> 4607, Book -> 22507, Tools_and_Home_Improvement -> 1926, Sports_and_Outdoor -> 3269, CDs_and_Vinyl -> 3749, Home_and_Kitche -> 4254, Apps_for_Android -> 2638, Office_Product -> 1243, Health_and_Personal_Care -> 2982, Cell_Phones_and_Accessorie -> 3447, Digital_Music -> 836, Toys_and_Game -> 2253, Beauty -> 2023, Patio_Lawn_and_Garde -> 994)


In [18]:
// Broadcast totalCategory for use in transformations
val totalCategoryBroadcast = sc.broadcast(totalCategory)

totalCategoryBroadcast: org.apache.spark.broadcast.Broadcast[scala.collection.Map[String,Int]] = Broadcast(11)


In [19]:
totalCategoryBroadcast

res3: org.apache.spark.broadcast.Broadcast[scala.collection.Map[String,Int]] = Broadcast(11)


In [20]:
val totalCategoryImmutable = totalCategory.mapValues(identity).toMap

totalCategoryImmutable: scala.collection.immutable.Map[String,Int] = Map(Patio_Lawn_and_Garde -> 994, Movies_and_TV -> 4607, Electronic -> 7825, Office_Product -> 1243, Tools_and_Home_Improvement -> 1926, Kindle_Store -> 3205, Home_and_Kitche -> 4254, Digital_Music -> 836, Automotive -> 1374, Grocery_and_Gourmet_Food -> 1297, Baby -> 916, Book -> 22507, Clothing_Shoes_and_Jewelry -> 5749, Toys_and_Game -> 2253, Health_and_Personal_Care -> 2982, Sports_and_Outdoor -> 3269, Beauty -> 2023, CDs_and_Vinyl -> 3749, Musical_Instrument -> 500, Cell_Phones_and_Accessorie -> 3447, Apps_for_Android -> 2638, Pet_Supplie -> 1235)


In [21]:
// Define a function to calculate c and d
def calculateCD(category: String, a: Int, b: Int, totalCount: Long, totalCategory: Map[String, Int]): (Int, Int) = {
  val cTotal = totalCategory(category)
  val c = cTotal - a
  val d = totalCount.toInt - a - b - c
  (c, d)
}

// Step 4: Add c and d to termCategoriesRDD
val termCategoriesWithCD = termCategoriesRDD.map { case ((category, term), (a, b)) =>
  val (c, d) = calculateCD(category, a, b, totalCount, totalCategoryImmutable)
  ((category, term), (a, b, c, d))
}

org.apache.spark.SparkException:  Task not serializable